In [26]:
import os
import re
import pandas as pd
import numpy as np

import statsmodels.api as sm
import warnings
import math
from textblob import TextBlob
warnings.filterwarnings("ignore")

In [12]:
df_zara = pd.read_csv(os.path.join(os.getcwd(), 'data', 'Reviews_Zara.csv'))
df_zara_fin = pd.read_excel(os.path.join(os.getcwd(), 'data', 'zara-year-wise-financials.xlsx'))

In [9]:
df_zara_reviews= df_zara.groupby('Year').size().reset_index(name='Number of Reviews')

In [10]:
df_zara_reviews

,Year,Number of Reviews
0,2012,9
1,2013,128
2,2014,283
3,2015,326
4,2016,571
5,2017,1244
6,2018,1381
7,2019,1695
8,2020,2931
9,2021,3297


In [14]:
yearly_average_ratings = df_zara.groupby(df_zara.columns[0])[df_zara.columns[2]].mean()
df_zara_avg_rating = yearly_average_ratings.reset_index()
df_zara_avg_rating.columns = ['Year', 'Average Rating']

In [15]:
df_zara_data = df_zara_fin.merge(df_zara_reviews, on='Year', how='outer' )

In [16]:
df_zara_data = df_zara_data.merge(df_zara_avg_rating, on ='Year', how = 'outer')

In [17]:
df_zara_data.fillna(0, inplace=True)

In [18]:
df_zara_data.drop(index=14, axis=1, inplace=True) 

In [19]:
df_zara_data.corr()

,Year,Zara-Revenue,Number of Reviews,Average Rating
Year,1.000000,0.945399,0.910324,0.556596
Zara-Revenue,0.945399,1.000000,0.782407,0.615751
Number of Reviews,0.910324,0.782407,1.000000,0.271274
Average Rating,0.556596,0.615751,0.271274,1.000000


In [20]:
y = df_zara_data['Zara-Revenue']
x = df_zara_data[['Number of Reviews', 'Average Rating']]
y = np.log(y)
x['Number of Reviews'] = np.log(x['Number of Reviews'] +1)

In [21]:
X= sm.add_constant(x)
log_log_reg = sm.OLS(y, X).fit()

In [22]:
log_log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Zara-Revenue   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.842
Method:                 Least Squares   F-statistic:                     35.69
Date:                Tue, 03 Oct 2023   Prob (F-statistic):           1.55e-05
Time:                        18:14:03   Log-Likelihood:                 14.465
No. Observations:                  14   AIC:                            -22.93
Df Residuals:                      11   BIC:                            -21.01
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 9.7236      0.056    174.903      0.000       9.601       9.846
Number of Reviews     0.0791      0.014      5.807      0.000       0.049       0.109
Average Rating       -0.0160      0.026     -0.604      0.558      -0.074       0.042
==============================================================================
Omnibus:                        0.993   Durbin-Watson:                   1.497
Prob(Omnibus):                  0.609   Jarque-Bera (JB):                0.718
Skew:                          -0.145   Prob(JB):                        0.698
Kurtosis:                       1.929   Cond. No.                         14.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
df_zara_data.to_excel(os.path.join(os.getcwd(), 'zara-data.xlsx'), index = False)